In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import tensorflow as tf

In [109]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource

In [130]:
dataset1 = DavisDataset.named_dataset("poster_translation")
dataset2 = DavisDataset.named_dataset("shapes_translation")

In [131]:
source1 = EventSource(dataset1, 
                      testing_fraction=0.2,
                      validation_fraction=0.4,
                      events_per_block=1000)
source2 = EventSource(dataset2, 
                      testing_fraction=0.2,
                      validation_fraction=0.4,
                      events_per_block=1000)

In [134]:
combined = source1 + source2

In [150]:
N_EVENT_SLICES = 5
EVENT_PIXEL_SCALING = 2
BATCH_SIZE = 60
N_EVENT_ROWS = 180 // EVENT_PIXEL_SCALING
N_EVENT_COLS = 240 // EVENT_PIXEL_SCALING

In [135]:
# Copied from the tensorflow tutorial in class
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def init_bias(shape):
    return tf.Variable(tf.zeros(shape))

In [181]:
def training_batch(training, batch_size):
    iterator = iter(training)
    blocks = [next(iterator) for i in range(batch_size)]
    data = np.stack([block.events_rescaled(n_layers=N_EVENT_SLICES,
                                         scaling=EVENT_PIXEL_SCALING) for block in blocks])
    labels = np.stack([block.delta_position for block in blocks])
    return data, labels
    
def test_data(source):
    data = np.stack([block.events_rescaled(n_layers=N_EVENT_SLICES,
                                         scaling=EVENT_PIXEL_SCALING) for block in source.testing()])
    labels = np.stack([block.delta_position for block in source.testing()])
    return data, labels

In [182]:
def n_output_channels(layer):
    return layer.get_shape()[-1].value

def n_outputs(layer):
    out = layer.get_shape()[1].value
    for d in layer.get_shape()[2:]:
        out *= d.value
    return out


In [185]:
input_tensor = tf.placeholder(tf.float32, [None, N_EVENT_COLS, N_EVENT_ROWS, N_EVENT_SLICES, 1])
dropout_keep_prob = tf.placeholder(tf.float32)

conv1 = tf.nn.conv3d(input_tensor, init_weights([3,3,3,1,14]), strides=[1,1,1,1,1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, init_bias(n_output_channels(conv1))))
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

conv2 = tf.nn.conv3d(pool1, 
                     init_weights([3,3,3,output_channels(pool1),14]), 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, init_bias(n_output_channels(conv2))))
pool2 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

fc3 = tf.reshape(pool2, [-1, n_outputs(pool2)])
fc3 = tf.add(tf.matmul(fc3, init_weights([n_outputs(pool2), 32])), init_bias(32))
fc3 = tf.nn.relu(fc3)
fc3 = tf.nn.dropout(fc3, dropout_keep_prob)

out = tf.add(tf.matmul(fc3, init_weights([n_outputs(fc3), 3])), init_bias(3))

y = tf.placeholder(tf.float32, [None, 3])

loss = tf.nn.l2_loss(tf.sub(out, y))
avg_loss = tf.reduce_mean(loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

init = tf.initialize_all_variables()

In [187]:
TRAINING_STEPS = 10

sess = tf.Session()
sess.run(init)

for step in range(TRAINING_STEPS):
    training_data = combined.training()  # reset the generator
    for i in range(combined.num_training // BATCH_SIZE):
        data_batch, labels_batch = training_batch(training_data, BATCH_SIZE)
        sess.run(train_optimizer, feed_dict={input_tensor: data_batch,
                                             y: labels_batch,
                                             dropout_keep_prob: 0.5})

data_test, labels_test = test_set()
avg = sess.run(avg_loss, feed_dict={input_tensor: data_test,
                                    y: labels_test,
                                    dropout_keep_prob: 1.0})
avg

KeyboardInterrupt: 